In [ ]:
############ combination of solution to solve more #########

Acknowledgment: All credit goes to those notebook
This notebook combines my original work with adapted components and ideas inspired by  public Kaggle notebooks output solution used here for ensembling:

https://www.kaggle.com/code/bibanh/qwen2-5-32b-arc-local-score

https://www.kaggle.com/code/seshurajup/code-golf-ensemble-local-score-391-400-dsl

solved-127-problems-local-pleaseupvote

https://www.kaggle.com/code/jacekwl/a-bit-more-of-code-golf

ineurips-2025-google-code-golf-championship1

liah-submission-gcgc-v2


I have restructured and integrated the techniques into my own pipeline, with modifications to logic, patterns, and implementation. Much respect to both authors for their contributions Use solution here combined.

In [ ]:
import os, json, zipfile
from collections import deque, Counter
import numpy as np
from tqdm import tqdm
from rich import print as print_rich
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

DIR = "/kaggle"

# ---------------- PRIMITIVES ----------------

def rot90_np(g): return np.rot90(g, 1)
def rot180_np(g): return np.rot90(g, 2)
def rot270_np(g): return np.rot90(g, 3)
def fliph_np(g): return np.fliplr(g)
def flipv_np(g): return np.flipud(g)
def invert_colors_np(g): return 9 - g
def identity_np(g): return g
def remap_colors_np(g):
    flat = g.flatten()
    uniq = np.unique(flat)
    mapping = {c: i for i, c in enumerate(uniq)}
    return np.vectorize(mapping.get)(g)

def majority_fill_np(g):
    vals, counts = np.unique(g, return_counts=True)
    fill_val = vals[np.argmax(counts)]
    out = g.copy()
    out[out == 0] = fill_val
    return out

primitive_code_snippets = {
    "identity": """return g""",
    "rot90": """return [list(row) for row in zip(*g[::-1])]""",
    "rot180": """return [row[::-1] for row in g[::-1]]""",
    "rot270": """return [list(row) for row in zip(*g)][::-1]""",
    "fliph": """return [row[::-1] for row in g]""",
    "flipv": """return g[::-1]""",
    "invert_colors": """return [[9 - c for c in row] for row in g]""",
    "remap_colors": """uniq = sorted(set(c for row in g for c in row)); mp = {c:i for i,c in enumerate(uniq)}; return [[mp[c] for c in row] for row in g]""",
    "majority_fill": """flat = [c for row in g for c in row]; fill_val = max(set(flat), key=flat.count); return [[fill_val if c==0 else c for c in row] for row in g]"""
}

# ---------------- TRANSFORM CLASS ----------------
class Transform:
    def __init__(self, func, name=None):
        self.func = func
        self.name = name or func.__name__
    def __call__(self, grid):
        try: return self.func(grid)
        except: return None
    def __repr__(self): return f"Transform({self.name})"
    def compose(self, other):
        def composed(g):
            r = self(g)
            return None if r is None else other(r)
        return Transform(composed, f"{other.name}∘{self.name}")

# ---------------- SOLVER ----------------
class ARCCombinatorialSolver:
    def __init__(self, max_depth=3, max_candidates=7000):
        self.max_depth = max_depth
        self.max_candidates = max_candidates
        self.learned_rules = {}
        self.primitives = [
            Transform(identity_np,"identity"),
            Transform(rot90_np,"rot90"),
            Transform(rot180_np,"rot180"),
            Transform(rot270_np,"rot270"),
            Transform(fliph_np,"fliph"),
            Transform(flipv_np,"flipv"),
            Transform(invert_colors_np,"invert_colors"),
            Transform(remap_colors_np,"remap_colors"),
            Transform(majority_fill_np,"majority_fill")
        ]

    def match(self, rule, pairs):
        return all((pred:=rule(inp)) is not None and np.array_equal(pred, out) for inp,out in pairs)

    def score_rule(self, rule, pairs):
        """Count exact matches (higher better), penalize longer names."""
        matches = sum(np.array_equal(rule(inp), out) for inp,out in pairs)
        return matches, -len(rule.name)

    def bfs_search(self, pairs):
        q = deque([(Transform(identity_np,"identity"),0)])
        valid_rules = []
        visited_hashes=set()

        def grid_hash(g): return hash(g.tobytes())

        while q and len(valid_rules) < 5:
            rule, depth = q.popleft()
            if depth > self.max_depth: continue

            trans = []
            fail=False
            for inp,_ in pairs:
                out = rule(inp)
                if out is None: fail=True; break
                trans.append(out)
            if fail: continue

            h = tuple(grid_hash(g) for g in trans)
            if h in visited_hashes: continue
            visited_hashes.add(h)

            if self.match(rule, pairs):
                valid_rules.append(rule)
                continue

            if depth < self.max_depth:
                for prim in self.primitives:
                    q.append((rule.compose(prim), depth+1))
                    if len(q) > self.max_candidates: break
        if valid_rules:
            valid_rules.sort(key=lambda r: (-self.score_rule(r,pairs)[0], self.score_rule(r,pairs)[1]))
        return valid_rules

    def solve(self, task):
        tid = task.get('id')
        train_pairs=[(np.array(p['input'],int),np.array(p['output'],int)) for p in task['train']]
        test_inputs=[np.array(p['input'],int) for p in task['test']]

        if tid in self.learned_rules:
            rule = self.learned_rules[tid]
            if self.match(rule, train_pairs):
                return rule, [rule(inp).tolist() for inp in test_inputs]

        rules = self.bfs_search(train_pairs)
        if not rules:
            return Transform(identity_np,"identity"), [inp.tolist() for inp in test_inputs]
        best_rule=rules[0]
        self.learned_rules[tid]=best_rule
        return best_rule,[best_rule(inp).tolist() for inp in test_inputs]

# ---------------- CODE GENERATOR ----------------
def generate_code(transform):
    names = transform.name.split("∘")
    names = list(reversed(names))
    if names == ["identity"]: return "def p(g):\n    return g\n"
    code_lines=["def p(g):","    res = g"]
    for name in names:
        snippet = primitive_code_snippets.get(name,"return g")
        for line in snippet.strip().split("\n"):
            if "return" in line: line=line.replace("return","res =")
            code_lines.append("    "+line.strip())
    code_lines.append("    return res")
    return "\n".join(code_lines)

# ---------------- MAIN ----------------
print("Loading tasks...")
train_tasks={}
for i in tqdm(range(1,401)):
    tid=f"{i:03d}"
    with open(f"{DIR}/input/google-code-golf-2025/task{tid}.json") as f:
        t=json.load(f); t['id']=tid; train_tasks[tid]=t

solver=ARCCombinatorialSolver(max_depth=5,max_candidates=20000)

submission_dir=f"{DIR}/working/submissionfile"
os.makedirs(submission_dir,exist_ok=True)

solved=0
for tid,task in tqdm(train_tasks.items()):
    transform,preds=solver.solve(task)
    code=generate_code(transform)
    with open(f"{submission_dir}/task{tid}.py","w") as f:
        f.write(code)
    solved+=1

# print_rich(f"[green]Generated solutions for {solved} tasks[/green]")
# with zipfile.ZipFile(f"{DIR}/working/submisisonfile.zip","w") as z:
#     for i in range(1,401):
#         tid=f"{i:03d}"
#         z.write(f"{submission_dir}/task{tid}.py",arcname=f"task{tid}.py")
# print_rich(f"[green]Submission zip created[/green]")


In [ ]:
import sys
sys.path.append('/kaggle/input/arc-generator-predifine-dict-code')
import re 
import ast 
from predefined_solution import *
from simple_arc_solution import *
from clean import *

def generate_general_fallback_solution(task_data):
    """
    Try simple general transformations and see if any solve the task.
    """
    from copy import deepcopy

    def all_match(task, func):
        for pair in task["train"] + task["test"] + task.get("arc-gen", []):
            try:
                output = func(pair["input"])
                if output != pair["output"]:
                    return False
            except:
                return False
        return True

    candidates = [
        ("identity", lambda g: g),
        ("transpose", lambda g: [list(row) for row in zip(*g)]),
        ("hflip", lambda g: [row[::-1] for row in g]),
        ("vflip", lambda g: g[::-1]),
        ("rotate180", lambda g: [row[::-1] for row in g[::-1]])
    ]

    for name, fn in candidates:
        if all_match(task_data, fn):
            print_rich(f"[green]Task solved by general fallback: {name}[/green]")
            # generate code string for fn
            if name == "identity":
                return "def p(g): return g"
            elif name == "transpose":
                return "def p(g): return [list(r) for r in zip(*g)]"
            elif name == "hflip":
                return "def p(g): return [r[::-1] for r in g]"
            elif name == "vflip":
                return "def p(g): return g[::-1]"
            elif name == "rotate180":
                return "def p(g): return [r[::-1] for r in g[::-1]]"

    return None

arc_generator  =  ARCSolutionGenerator()

In [ ]:
import os
import json
import zipfile
import copy
import re
import math
import ast
import string
from collections import Counter
from functools import reduce
from typing import List, Tuple

from tqdm import tqdm
from rich import print as print_rich
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

DIR = "/kaggle"


def check_solution(solution, task_data):
    try:
        namespace = {}
        exec(solution, namespace)
        if 'p' not in namespace:
            return False
        all_examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        for example in all_examples:
            input_grid = copy.deepcopy(example['input'])
            expected = example['output']
            try:
                actual = namespace['p'](input_grid)
                if actual != expected:
                    return False
            except Exception:
                return False
        return True
    except Exception:
        return False


submissions = [
    "/kaggle/input/d/muhammadqasimshabbir/oh-barnacles/submission",
    "/kaggle/input/solved-127-problems-local-pleaseupvote-submission/submission",
    "/kaggle/input/neurips-2025-google-code-golf-championship1",
    "/kaggle/input/liah-submission-gcgc-v2",
    "/kaggle/input/code-golf-ensemble-local-score-391-400-dsl",
    "/kaggle/working/submisisonfile",
]

simple_solution = """def p(g):return g"""

solved = 0
total_score = 0
os.makedirs(f"{DIR}/working/submission", exist_ok=True)
removed_by_method = Counter()

for task_num in tqdm(range(1, 401)):
    task_id = f"{task_num:03d}"
    solutions = []
    task_data_path = f"{DIR}/input/google-code-golf-2025/task{task_id}.json"
    task_data = json.load(open(task_data_path))

    is_solved = False
    
    # First try predefined solutions if available
    if task_num in PREDEFINED_SOLUTIONS:
        solution_code = PREDEFINED_SOLUTIONS[task_num]
        if check_solution(solution_code, task_data):
            solutions.append(solution_code)
            is_solved = True
            print_rich(f"[yellow]{task_id} - solved by predefined solution dict[/yellow]")
    
    # Then try existing submissions
    if not is_solved:
        for submission_path in submissions:
            task_code_path = f"{submission_path}/task{task_id}.py"
            if not os.path.exists(task_code_path):
                continue
            with open(task_code_path, 'r') as f:
                solution_code = f.read()
            if check_solution(solution_code, task_data):
                solutions.append(solution_code)
                is_solved = True
            print_rich(f"[green]{task_id} - solved by sumissions  [/green]")
            
    # If not solved by predefined or existing submissions, try the ARC generator
    if not is_solved:
        try:
            generated_solution = arc_generator.generate_solution(task_data)
            if arc_generator.verify_solution(generated_solution, task_data):
                solutions.append(generated_solution)
                is_solved = True
                print_rich(f"[yellow]{task_id} - solved by generator[/yellow]")
        except Exception as e:
            print_rich(f"[red]Error generating solution for task {task_id}: {str(e)}[/red]")
            pass
            
    # Fallback: try all predefined solutions if none worked
    if not is_solved:
        for key, sol_code in PREDEFINED_SOLUTIONS.items():
            if check_solution(sol_code, task_data):
                        solutions.append(sol_code)
                        is_solved = True
                        print_rich(f"[magenta]{task_id} - solved by global predefined dict (task {key})[/magenta]")
                        break
    if not is_solved:
        general_code = generate_general_fallback_solution(task_data)
        print_rich(f"[red]{task_id} - solved by general fallback[red]")
        if general_code and check_solution(general_code, task_data):
            solutions.append(general_code)
            is_solved = True

    score = 0.001
    best_solution = simple_solution

    if solutions:
        # Apply all minimization techniques to each solution
        minimized_solutions = []
        for solution in solutions:
            for method in [remove_spaces, minimize_indentation,
                         substitute_enumerate, substitute_range,
                         join_block_lines, strip_trailing_whitespaces,
                         remove_empty_lines, remove_comments,
                         shorten_variable_names, def_to_lambda, remove_spaces]:
                b1 = get_bytes(solution)
                new_solution = solution
                try:
                    new_solution = method(solution)
                except Exception as e:
                    print(e)
                b2 = get_bytes(new_solution)

                if check_solution(new_solution, task_data) and b2 < b1:
                    solution = new_solution
                    removed_by_method[method] += (b1 - b2)
            
            minimized_solutions.append(solution)
        
        best_solution = min(minimized_solutions, key=get_bytes)
        score = calculate_score(best_solution)
        if is_solved:
            solved += 1

    total_score += score

    with open(f"{DIR}/working/submission/task{task_id}.py", "w") as f:
        f.write(best_solution)

# Create the submission zip
with zipfile.ZipFile(f"{DIR}/working/submission.zip", "w") as zipf:
    for task_num in range(1, 401):
        task_id = f"{task_num:03d}"
        zipf.write(f"{DIR}/working/submission/task{task_id}.py", 
                  arcname=f"task{task_id}.py")

print_rich(f"[green]Total solved: {solved} / 400[/green]")
print_rich(f"[blue]LB Score: {total_score:.3f}[/blue]")

for k, v in removed_by_method.most_common():
    print(f"{k.__name__:<30}{v:>5}")